In [1]:
import pandas as pd
import numpy as np
import tabula
import re
pd.set_option('display.max_rows', 500)

In [2]:
dft_2021 = tabula.read_pdf('2021 Blog.pdf', pages='all', stream=True, pandas_options={'header': None})
dft_2022 = tabula.read_pdf('2022 12 Patch Day Blog V9.0.pdf', pages='all', stream=True, pandas_options={'header': None})
dft_2023 = tabula.read_pdf('2023 12 Patch Day Blog V2.0 (1).pdf', pages='all', stream=True, pandas_options={'header': None})

Sep 26, 2024 12:42:06 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 26, 2024 12:42:06 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 26, 2024 12:42:07 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 26, 2024 12:42:07 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 26, 2024 12:42:07 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Sep 26, 2024 12:42:07 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>


In [3]:
def newHeader(xdf):
    data_list = []
    header = xdf[0].iloc[0]
    xdf[0].columns = header
    xdf[0] = xdf[0].drop([0])
    xdf[0]["cve_id"] = xdf[0]["Title"].str.extract(r'(CVE-....-\d+)')
    for i in range(0, len(xdf)):
        if i == 0:
            data = xdf[i].iloc[1:]
        else:
            data = xdf[i]
        if data.shape[1] == header.shape[0]:
            data.columns = header
            xdf[i]["cve_id"] = xdf[i]["Title"].str.extract(r'(CVE-....-\d+)')
            data_list.append(data)
    return data_list

In [4]:
dftt_2021 = newHeader(dft_2021)
dftt_2022 = newHeader(dft_2022)
dftt_2023 = newHeader(dft_2023)

In [5]:
sap_2021 = pd.concat(dftt_2021)
sap_2022 = pd.concat(dftt_2022)
sap_2023 = pd.concat(dftt_2023)

In [6]:
# Pandas merge rows in a column 'Title'
# https://stackoverflow.com/questions/64141451/pandas-merge-rows-in-a-column
def mergeTitle(ydf):
    ydf['Title'] = ydf['Title'].astype(str)
    blocks = ydf['CVSS'].notna().cumsum()
    agg_dict = {col:' '.join if col=='Title' else 'first' for col in ydf}
    df_t = ydf.groupby(blocks).agg(agg_dict).reset_index(drop=True)
    df_t.dropna(inplace=True)
    df_t = df_t[(df_t['Note#'] != 'Note#')]
    df_t['Note#'] = df_t['Note#'].astype(int)
    df_t.reset_index(drop=True, inplace=True)
    return df_t

In [7]:
sap_2021_notes = mergeTitle(sap_2021)
sap_2022_notes = mergeTitle(sap_2022)
sap_2023_notes = mergeTitle(sap_2023)

In [8]:
#sap_2023_notes

In [9]:
# SAP Notes 2024
#sap_2024_notes = pd.read_csv('sap_notes_2024.csv')

In [10]:
sap_202401 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/january-2024.html',
                 flavor='html5lib')
sap_202402 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/february-2024.html',
                 flavor='html5lib')
sap_202403 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/march-2024.html',
                 flavor='html5lib')
sap_202404 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/april-2024.html',
                 flavor='html5lib')
sap_202405 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/may-2024.html',
                 flavor='html5lib')
sap_202406 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/june-2024.html',
                 flavor='html5lib')
sap_202407 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/july-2024.html',
                 flavor='html5lib')
sap_202408 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/august-2024.html',
                 flavor='html5lib')
sap_202409 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/september-2024.html',
                 flavor='html5lib')

In [11]:
def etData1(x):
    _df = pd.DataFrame(x[0], columns=['Note#', 'Title', 'Severity', 'CVSS'])
    _df.rename(columns={'Severity': 'Priority'}, inplace=True)
    _df["cve_id"] = _df["Title"].str.extract(r'(CVE-....-\d+)')
    return _df

def etData2(x):
    _df = pd.DataFrame(x[0], columns=['Note#', 'Title', 'Priority', 'CVSS'])
    _df["cve_id"] = _df["Title"].str.extract(r'(CVE-....-\d+)')
    return _df

In [12]:
sap_202401_df = etData1(sap_202401)
sap_202402_df = etData1(sap_202402)
sap_202403_df = etData1(sap_202403)
sap_202404_df = etData1(sap_202404)
sap_202405_df = etData1(sap_202405)
sap_202406_df = etData1(sap_202406)
sap_202407_df = etData2(sap_202407)
sap_202408_df = etData2(sap_202408)
sap_202409_df = etData2(sap_202409)

In [13]:
sap_2024_ls = [sap_202401_df,sap_202402_df,sap_202403_df,sap_202404_df,sap_202405_df,sap_202406_df,sap_202407_df,sap_202408_df,sap_202409_df]
sap_2024_df = pd.concat(sap_2024_ls)

In [14]:
sap_2024_notes = sap_2024_df.dropna()
#sap_2024_notes = pd.read_csv('sap_notes_2024.csv')

In [15]:
# start extract cve_ids string for terminal commands
# Mejorar y crear un file para flag del commando.
l_sap_cve = sap_2022_notes.cve_id.to_list()

In [16]:
clean_sap_cve = [x for x in l_sap_cve if str(x) != 'nan']

In [17]:
#cleanedList

In [ ]:
string_list = [str(element) for element in clean_sap_cve]
delimiter = " "
result_string_cve = delimiter.join(string_list)

In [ ]:
result_string_cve

In [ ]:
# end extract cve_ids

In [18]:
### Crear DataFrame de output Sploitscan
def dataframeSplotscan(file_json):
    data = pd.DataFrame(columns = ['cve_id', 'dateUpdated', 'descriptions', 'product_l', 'epss_l', 'percentile', 'priority_l', 'cweId'])
    dict_list = []
    sap_sp = pd.read_json(f'{file_json}', typ='series')
    for i in sap_sp:
        if 'problemTypes' in i['CVE Data']['containers']['cna'].keys():            
            if 'cweId' in i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0].keys():
                cweId = i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0]['cweId']
            else:
                cweId = i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0]['description']
        else:
            cweId = None
        if len(i['EPSS Data']['data']) == 1:
            epss_l = i['EPSS Data']['data'][0]['epss']
            percentile = i['EPSS Data']['data'][0]['percentile']
        else:
            epss_l = None
            percentile = None
        if 'datePublished' in i['CVE Data']['cveMetadata']:
            datePublished = i['CVE Data']['cveMetadata']['datePublished']
        else:
            datePublished = None
        if 'references' in i['CVE Data']['containers']['cna'].keys():            
            note_id = re.findall('[2,3]{1}[0-9]{6}', str(i['CVE Data']['containers']['cna']['references'][0]['url']))
        else:
            note_id = None
        row_list = {'cve_id': i['CVE Data']['cveMetadata']['cveId'],
                    #'datePublished': i['CVE Data']['cveMetadata']['datePublished'],
                    'datePublished': datePublished,
                    'dateUpdated': i['CVE Data']['cveMetadata']['dateUpdated'],
                    'descriptions': i['CVE Data']['containers']['cna']['descriptions'][0]['value'],
                    'product_l': i['CVE Data']['containers']['cna']['affected'][0]['product'],
                    #'epss_l': i['EPSS Data']['data'][0]['epss'],
                    #'benchmark': i['VulnCheck Data']['_benchmark'],
                    'epss_l': epss_l,
                    'percentile': percentile,
                    'priority_l': i['Priority']['Priority'],
                    'cweId': cweId,
                    'note_id': str(note_id)}
        dict_list.append(row_list)

    data = pd.DataFrame.from_dict(dict_list)
    return(data)

In [19]:
#especial 2021 sap notes
#sp_sap_cve_2021 = dataframeSplotscan('sap_history_data_curate/sp_2021_last.json')

In [20]:
sp_sap_cve_2021 = dataframeSplotscan('sap_history_data_curate/sp_2021_last.json')
sp_sap_cve_2021.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2022 = dataframeSplotscan('sap_history_data_curate/sp_2022_last.json')
sp_sap_cve_2022.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2023 = dataframeSplotscan('sap_history_data_curate/sp_2023_last.json')
sp_sap_cve_2023.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2024 = dataframeSplotscan('sap_history_data_curate/sp_sap_2024.json')
sp_sap_cve_2024.drop_duplicates(subset=['cve_id'], inplace=True)

In [21]:
cp_sap_cve_2021 = pd.read_csv('sap_history_data_curate/sap_2021_clean.csv')
cp_sap_cve_2022 = pd.read_csv('sap_history_data_curate/sap_2022_clean.csv')
cp_sap_cve_2023 = pd.read_csv('sap_history_data_curate/sap_2023_clean.csv')
cp_sap_cve_2024 = pd.read_csv('sap_history_data_curate/cveprio_sap_2024.csv')

In [22]:
cp_sap_cve_2021.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2022.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2023.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2024.drop_duplicates(subset=['cve_id'], inplace=True)

In [23]:
sap_cve_2023 = sap_2023_notes.merge(sp_sap_cve_2023, on=['cve_id'])
sap_cve_2023 = sap_cve_2023.merge(cp_sap_cve_2023, on=['cve_id'])
sap_cve_2023['sap_note_year'] = '2023'

In [24]:
sap_cve_2022 = sp_sap_cve_2022.merge(sap_2022_notes, on=['cve_id'])
sap_cve_2022 = sap_cve_2022.merge(cp_sap_cve_2022, on=['cve_id'])
sap_cve_2022['sap_note_year'] = '2022'

In [25]:
sap_cve_2021 = sp_sap_cve_2021.merge(sap_2021_notes, on=['cve_id'])
sap_cve_2021 = sap_cve_2021.merge(cp_sap_cve_2021, on=['cve_id'])
sap_cve_2021['sap_note_year'] = '2021'

In [26]:
sap_cve_2024 = sp_sap_cve_2024.merge(sap_2024_notes, on=['cve_id'])
sap_cve_2024 = sap_cve_2024.merge(cp_sap_cve_2024, on=['cve_id'])
sap_cve_2024['sap_note_year'] = '2024'

In [27]:
#sap_2024_notes.info()

In [28]:
#sap_cve_2024.info()

In [29]:
#sap_cve_2024

In [60]:
sap_cve_df = pd.concat([sap_cve_2021,sap_cve_2022,sap_cve_2023,sap_cve_2024])

In [61]:
#sap_cve_df['epss'] = sap_cve_df['epss'].map(lambda x: x * 100).astype('float').round(2)

In [62]:
#sap_cve_df['cveInfo'] = sap_cve_df['cve_id'].apply(lambda x: x.replace(f'{x}', f'https://www.cvedetails.com/cve/{x}/'))

In [63]:
#sap_cve_df.drop_duplicates(subset=['Note#'], inplace=True)

In [64]:
sap_cve_df.loc[(sap_cve_df['Priority'] == 'Hot') | (sap_cve_df['Priority'] == 'HotNews') | (sap_cve_df['Priority'] == 'Very High'), 'Priority'] = 'Hot News'
#sap_cve_df.loc[(sap_cve_df['Priority'] == 'Hot') | (sap_cve_df['Priority'] == 'HotNews'), 'Priority'] = 'Hot News'

In [65]:
sap_cve_df.loc[(sap_cve_df['cweId'] == 'Cross-Site Scripting') | (sap_cve_df['cweId'] == 'Cross Site Scripting') | (sap_cve_df['cweId'] == "Cross-Site Scripting (XSS)"), 'cweId'] = "CWE-79"

In [66]:
sap_cve_df['cweId'].unique()

array(['Denial of Service', 'Missing Authorization Check',
       'Information Disclosure', 'Header Manipulation', 'CWE-79',
       'Missing Encryption', 'Improper Input Validation',
       'XML External Entity Reference (XXE)', 'Remote Code Execution',
       'SQL Injection', 'Missing Authorization',
       'Weak Password Requirements', 'Clickjacking', 'Tabnabbing',
       'Other', 'Improper Authentication', 'Directory Traversal',
       'Missing Authentication Check', 'Code Execution',
       'Authentication Bypass', 'Missing Authorization check',
       'Insecure Deserialization', 'Server-Side Request Forgery',
       'Reverse Tabnabbing', 'Code Injection', 'Information disclosure',
       'Unquoted Search Path or Element (CWE - 428)', 'CWE-284',
       'CWE-611', 'CWE-290', 'Missing XML Validation',
       'URL Redirection to Untrusted Site',
       'Missing XML Validation (CWE - 112)', 'CWE-125', 'CWE-476',
       'CWE-787',
       "Improper Neutralization of Special Elements in O

In [67]:
sap_cve_df.to_csv('sap_cve_last_02.csv', index=False)

In [68]:
sap_cve_df[['epss', 'Note#', 'cweId']]

,epss,Note#,cweId
0,0.00124,3000306,Denial of Service
1,0.00124,3000306,Denial of Service
2,0.00054,2863397,Missing Authorization Check
3,0.00065,2826528,Information Disclosure
4,0.00054,2984034,Header Manipulation
...,...,...,...
119,0.00043,3437585,CWE-213
120,0.00045,3505293,CWE-862
121,0.00043,2256627,CWE-862
122,0.00045,3496410,CWE-862
